In [1]:
configs = { 
"fs.azure.account.auth.type": "CustomAccessToken",
"fs.azure.account.custom.token.provider.class": spark.conf.get("spark.databricks.passthrough.adls.gen2.tokenProviderClassName")
}

In [2]:
filesystems = ['bronze', 'silver', 'gold']
lakeName = 'devactsoftdatalake'
subDirectory = ''

In [3]:
#dbutils.fs.unmount('/mnt/%s' % (filesystems[1]))

In [4]:
def Diff(li1, li2): 
    return (list(set(li1) - set(li2)))
def makefilter(mntName):
   def iterator(x):
      return x.mountPoint == '/mnt/%s' % (mntName) 
   return iterator
def getMountedZones():
  mounted = []
  for mnt in filesystems:
    result = list(filter(makefilter(mnt), dbutils.fs.mounts()))
    if (len(result) == 1):
        mounted.append(mnt)
  return mounted  
def mount(filesystem):
  dbutils.fs.mount(
    source = "abfss://%s@%s.dfs.core.windows.net/%s" % (filesystem, lakeName, subDirectory),
    mount_point = "/mnt/%s" % (filesystem),
    extra_configs = configs)
  print('Mounted /mnt/%s' % (filesystem))

#Diff(filesystems, getMountedZones())


In [5]:
zonesToMnt = Diff(filesystems, getMountedZones())

if (len(zonesToMnt) == 0):
  print('All Zones already mounted')
else:
  for zone in zonesToMnt:
    mount(zone)
  



Mounted /mnt/silver

In [6]:
dbutils.fs.mounts()

Out[24]: [MountInfo(mountPoint='/mnt/training', source='wasbs://training@dbtraineastus2.blob.core.windows.net/', encryptionType=''),
 MountInfo(mountPoint='/databricks-datasets', source='databricks-datasets', encryptionType=''),
 MountInfo(mountPoint='/mnt/sandboxes', source='abfss://sandboxes@devactsoftdatalake.dfs.core.windows.net/', encryptionType=''),
 MountInfo(mountPoint='/mnt/gold', source='abfss://gold@devactsoftdatalake.dfs.core.windows.net/', encryptionType=''),
 MountInfo(mountPoint='/mnt/silver', source='abfss://silver@devactsoftdatalake.dfs.core.windows.net/', encryptionType=''),
 MountInfo(mountPoint='/mnt/SandboxStorage', source='abfss://sandboxes@devactsoftdatalake.dfs.core.windows.net/', encryptionType=''),
 MountInfo(mountPoint='/mnt/adls', source='adl://warretest.azuredatalakestore.net/', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-tracking', source='databricks/mlflow-tracking', encryptionType=''),
 MountInfo(mountPoint='/databricks-results', source='databricks-results', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-registry', source='databricks/mlflow-registry', encryptionType=''),
 MountInfo(mountPoint='/mnt/bronze', source='abfss://bronze@devactsoftdatalake.dfs.core.windows.net/', encryptionType=''),
 MountInfo(mountPoint='/', source='DatabricksRoot', encryptionType='')]

In [7]:
dbutils.fs.ls("/mnt/bronze/Samples")

Out[32]: [FileInfo(path='dbfs:/mnt/bronze/Samples/ATT3ppNotifications/', name='ATT3ppNotifications/', size=0),
 FileInfo(path='dbfs:/mnt/bronze/Samples/ATT3ppNotificationsDelta/', name='ATT3ppNotificationsDelta/', size=0),
 FileInfo(path='dbfs:/mnt/bronze/Samples/BillingAccounts/', name='BillingAccounts/', size=0),
 FileInfo(path='dbfs:/mnt/bronze/Samples/BillingCompanies/', name='BillingCompanies/', size=0),
 FileInfo(path='dbfs:/mnt/bronze/Samples/CompanyModules/', name='CompanyModules/', size=0),
 FileInfo(path='dbfs:/mnt/bronze/Samples/Modules/', name='Modules/', size=0),
 FileInfo(path='dbfs:/mnt/bronze/Samples/PaymentInfo/', name='PaymentInfo/', size=0),
 FileInfo(path='dbfs:/mnt/bronze/Samples/ReportJobs/', name='ReportJobs/', size=0),
 FileInfo(path='dbfs:/mnt/bronze/Samples/Timekeeing1yr/', name='Timekeeing1yr/', size=0),
 FileInfo(path='dbfs:/mnt/bronze/Samples/TimekeeingFormData1yr/', name='TimekeeingFormData1yr/', size=0)]